# Q-LOCK Attractor Engine - Azure ML Demo

This notebook demonstrates end-to-end usage of the Q-LOCK Attractor Engine in Azure Machine Learning environments.

**Note:** This demo runs entirely locally using Qiskit Aer simulator. No Azure Quantum or QPU access is required.

## Features Demonstrated:
1. Package installation via pip
2. Running `baseline` mode (no watermarking)
3. Running `watermark` mode (identity-locked perturbations)
4. Running `fidelity` mode (calculate metrics)
5. Running `compare` mode (compare results)
6. Generating plots
7. Saving results to `runs/<timestamp>/` directory

## 1. Install Package and Dependencies

First, we'll install the Q-LOCK package in editable mode along with all dependencies.

In [ ]:
%%bash
# Install Q-LOCK package in editable mode
cd ..
pip install -e .
pip install jupyter matplotlib pandas

## 2. Setup Environment

Import necessary libraries and verify installation.

In [ ]:
import sys
import os
from datetime import datetime
from pathlib import Path

# Verify Qiskit installation
import qiskit
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print(f"Qiskit version: {qiskit.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Python version: {sys.version}")
print("\n✓ All dependencies loaded successfully!")
print("\n⚠️  Note: Running locally with Qiskit Aer simulator (no QPU access required)")

## 3. Create Output Directory with Timestamp

We'll create a timestamped directory under `runs/` to store all our results.

In [ ]:
# Create timestamped output directory in runs/
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
runs_base = Path("../runs")
output_dir = runs_base / timestamp
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {output_dir}")
print(f"Absolute path: {output_dir.resolve()}")

## 4. Define Example QASM Circuit

We'll use a simple GHZ-style circuit with rotation gates for testing.

In [ ]:
# Define a sample QASM circuit
sample_qasm = """
OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];

h q[0];
cx q[0], q[1];
cx q[1], q[2];

rz(0.3) q[0];
ry(0.6) q[1];
rx(0.9) q[2];

measure q -> c;
""".strip()

# Save circuit to file
circuit_file = output_dir / "test_circuit.qasm"
with open(circuit_file, "w") as f:
    f.write(sample_qasm)

print("Sample QASM circuit:")
print(sample_qasm)
print(f"\n✓ Saved to: {circuit_file}")

## 5. Run Baseline Mode

First, let's run the circuit without any watermarking to establish a baseline.

In [ ]:
# Convert Path objects to strings for shell compatibility
circuit_file_str = str(circuit_file)
output_dir_str = str(output_dir)

In [ ]:
%%bash -s "$circuit_file_str" "$output_dir_str"
# Run baseline mode
cd ..
python q_lock_cli.py baseline \
    --circuit "$1" \
    --shots 2048 \
    --output-dir "$2"

## 6. Run Watermark Mode

Now apply identity-locked watermarking to the circuit.

In [ ]:
# Define identity for watermarking
identity = "AzureML-Demo-User-2025"
print(f"Using identity: {identity}")

In [ ]:
%%bash -s "$circuit_file_str" "$output_dir_str" "$identity"
# Run watermark mode
cd ..
python q_lock_cli.py watermark \
    --circuit "$1" \
    --identity "$3" \
    --shots 2048 \
    --output-dir "$2"

## 7. Run Fidelity Mode

Calculate fidelity metrics comparing baseline and watermarked distributions.

In [ ]:
%%bash -s "$circuit_file_str" "$output_dir_str" "$identity"
# Run fidelity mode
cd ..
python q_lock_cli.py fidelity \
    --circuit "$1" \
    --identity "$3" \
    --shots 2048 \
    --output-dir "$2"

## 8. Run Compare Mode

Compare all results and generate summary tables.

In [ ]:
%%bash -s "$output_dir_str"
# Run compare mode
cd ..
python q_lock_cli.py compare \
    --output-dir "$1"

## 9. Load and Visualize Results

Now let's load the results and create visualizations.

In [ ]:
import json

# Find result files
baseline_files = sorted(output_dir.glob("baseline_*.json"))
watermark_files = sorted(output_dir.glob("watermark_*.json"))
fidelity_files = sorted(output_dir.glob("fidelity_*.json"))

print(f"Found {len(baseline_files)} baseline file(s)")
print(f"Found {len(watermark_files)} watermark file(s)")
print(f"Found {len(fidelity_files)} fidelity file(s)")

# Load most recent results
with open(baseline_files[-1], "r") as f:
    baseline_data = json.load(f)

with open(watermark_files[-1], "r") as f:
    watermark_data = json.load(f)

with open(fidelity_files[-1], "r") as f:
    fidelity_data = json.load(f)

print("\n✓ Loaded all result files")

In [ ]:
# Extract counts for visualization
baseline_counts = baseline_data["counts"]
watermark_counts = watermark_data["counts"]

# Get all unique states
all_states = sorted(set(baseline_counts.keys()) | set(watermark_counts.keys()))

# Prepare data for plotting
baseline_vals = [baseline_counts.get(s, 0) for s in all_states]
watermark_vals = [watermark_counts.get(s, 0) for s in all_states]

# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Baseline
ax1.bar(range(len(all_states)), baseline_vals, color='blue', alpha=0.7)
ax1.set_xlabel('Quantum State')
ax1.set_ylabel('Counts')
ax1.set_title('Baseline Distribution (No Watermark)')
ax1.set_xticks(range(len(all_states)))
ax1.set_xticklabels(all_states, rotation=45, ha='right')
ax1.grid(axis='y', alpha=0.3)

# Watermark
ax2.bar(range(len(all_states)), watermark_vals, color='red', alpha=0.7)
ax2.set_xlabel('Quantum State')
ax2.set_ylabel('Counts')
ax2.set_title('Watermarked Distribution (Identity-Locked)')
ax2.set_xticks(range(len(all_states)))
ax2.set_xticklabels(all_states, rotation=45, ha='right')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plot_file = output_dir / "distribution_comparison.png"
plt.savefig(plot_file, dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Saved plot: {plot_file}")

In [ ]:
# Display fidelity metrics
metrics = fidelity_data["metrics"]

print("\n" + "="*60)
print("FIDELITY METRICS SUMMARY")
print("="*60)
print(f"Total Variation Distance: {metrics['total_variation_distance']:.6f}")
print(f"KL Divergence:            {metrics['kl_divergence']:.6f}")
print(f"Hellinger Distance:       {metrics['hellinger_distance']:.6f}")
print("="*60)
print("\nInterpretation:")
print("  • TVD close to 0 means distributions are very similar")
print("  • Small KL divergence indicates high fidelity")
print("  • Hellinger distance ∈ [0,1] measures distributional similarity")

In [ ]:
# Create metrics visualization
fig, ax = plt.subplots(figsize=(10, 6))

metric_names = ['TVD', 'KL Divergence', 'Hellinger']
metric_values = [
    metrics['total_variation_distance'],
    metrics['kl_divergence'],
    metrics['hellinger_distance']
]

# Normalize for better visualization
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bars = ax.barh(metric_names, metric_values, color=colors, alpha=0.8)

ax.set_xlabel('Metric Value', fontsize=12)
ax.set_title('Fidelity Metrics: Baseline vs Watermarked', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, (bar, val) in enumerate(zip(bars, metric_values)):
    ax.text(val + max(metric_values)*0.01, i, f'{val:.6f}', va='center', fontsize=10)

plt.tight_layout()
metrics_plot = output_dir / "fidelity_metrics.png"
plt.savefig(metrics_plot, dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Saved metrics plot: {metrics_plot}")

## 10. Summary

List all generated files in the output directory.

In [ ]:
print("\n" + "="*60)
print(f"OUTPUT DIRECTORY: {output_dir}")
print("="*60)
print("\nGenerated files:")

for file in sorted(output_dir.iterdir()):
    size = file.stat().st_size
    print(f"  • {file.name:40s} ({size:>8,} bytes)")

print("\n✓ Demo complete!")
print(f"\nAll results saved to: {output_dir.resolve()}")
print("\n⚠️  Note: All simulations ran locally with Qiskit Aer (no cloud resources used)")

## Appendix: CLI Command Reference

For reference, here are the CLI commands used in this notebook:

```bash
# Installation
pip install -e .

# Baseline mode (no watermarking)
python q_lock_cli.py baseline \
    --circuit circuit.qasm \
    --shots 2048 \
    --output-dir ./runs/$(date +%Y%m%d_%H%M%S)

# Watermark mode (identity-locked perturbations)
python q_lock_cli.py watermark \
    --circuit circuit.qasm \
    --identity "your-identity" \
    --shots 2048 \
    --output-dir ./runs/$(date +%Y%m%d_%H%M%S)

# Fidelity mode (calculate metrics)
python q_lock_cli.py fidelity \
    --circuit circuit.qasm \
    --identity "your-identity" \
    --shots 2048 \
    --output-dir ./runs/$(date +%Y%m%d_%H%M%S)

# Compare mode (compare results)
python q_lock_cli.py compare \
    --output-dir ./runs/$(date +%Y%m%d_%H%M%S)
```

### Notes:
- All commands run locally using Qiskit Aer simulator
- No Azure Quantum or QPU access required
- No `az quantum` CLI dependency
- All outputs saved to local `runs/` directory
- Compatible with Azure ML compute instances